In [1]:
!git clone https://github.com/MeesLursen/KernelLTL.git

Cloning into 'KernelLTL'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 83 (delta 38), reused 60 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 42.79 KiB | 1.71 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [2]:
!cd /kaggle/working/KernelLTL && git pull && cd ..

Already up to date.


In [3]:
import sys
sys.path.insert(0, "/kaggle/working/KernelLTL") 

In [4]:
import math
import torch
from kernel_class_torch import LTLKernel_torch
import time

In [5]:
start = time.time()

T       = 20
AP      = 5
seed    = 1

eps     = 0.01
delta   = 1 - 0.99
N       = math.ceil((2 / eps**2) * math.log(2 / delta))

m       = 2000


kernel = LTLKernel_torch(T, AP, seed)
print(kernel.device)
kernel.sample_traces_kernel(N)
kernel.sample_formulas_kernel(m)
kernel.build_F()
mid = time.time()
print(mid - start)
kernel.build_K()


end = time.time()
print(end - start)

cuda
76.10323739051819
235.3087363243103


In [12]:
# --- Diagnostics ---
print(kernel.K.shape)
# print("K diag (should equal N):", tf.unique(tf.reshape(tf.round(tf.linalg.diag_part(kernel.K_tf)), [-1])))
# print("K symmetric?", tf.reduce_all(tf.equal(kernel.K_tf, tf.transpose(kernel.K_tf))))
print("K sample values:", kernel.K[:5,:5])
# print(kernel.formulas)

torch.Size([2000, 2000])
K sample values: tensor([[ 105967,    -363, -105967,      67,    -201],
        [   -363,  105967,     363,      53,     277],
        [-105967,     363,  105967,     -67,     201],
        [     67,      53,     -67,  105967,     -89],
        [   -201,     277,     201,     -89,  105967]], dtype=torch.int32)


In [19]:
print(kernel.formulas[0])
print(kernel.formulas[2])
print(kernel.K[0,2])
print(torch.unique(kernel.F[0,:], return_counts = True))
print(torch.unique(kernel.F[2,:], return_counts = True))

(G (~(p2)))
(F (X p4))
tensor(-105967, dtype=torch.int32)
(tensor([1], dtype=torch.int8), tensor([105967]))
(tensor([-1], dtype=torch.int8), tensor([105967]))
